In [ ]:
import os
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# Google API configuration
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
CLIENT_SECRETS_FILE = "client_secret.json"
TOKEN_FILE = "token.json"

def get_authenticated_service():
    """
    Authenticate with YouTube API and return a service object.
    """
    creds = None

    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                print("🔄 Token expired. Attempting to refresh...")
                creds.refresh(Request())
                print("✅ Token refreshed successfully!")
            except Exception as e:
                print(f"⚠️ Token refresh failed: {e}\n🌐 Opening browser for re-authentication...")
                creds = None  # Force re-authentication

        if creds is None:
            print("🌐 Opening browser for authentication...")
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        # ✅ Save the new token immediately
        with open(TOKEN_FILE, "w") as token_file:
            token_file.write(creds.to_json())

    return build("youtube", "v3", credentials=creds)

def upload_to_youtube(video_path, title, description, folder_path):
    """
    Uploads a video to YouTube and saves the video URL in 'youtube_link.txt'.
    """
    print(f"📤 Uploading: {title}...")
    youtube = get_authenticated_service()
    request = youtube.videos().insert(
        part="snippet,status",
        body={
            "snippet": {"title": title, "description": description, "categoryId": "27"},
            "status": {"privacyStatus": "unlisted"}
        },
        media_body=MediaFileUpload(video_path, chunksize=-1, resumable=True)
    )

    _, response = request.next_chunk()
    video_url = f"https://www.youtube.com/watch?v={response['id']}"
    with open(os.path.join(folder_path, "youtube_link.txt"), "w") as f:
        f.write(video_url + "\n")

    print(f"✅ Upload complete! Video URL: {video_url}")
    return video_url

if __name__ == "__main__":
    parent_folder = input("📂 Enter the folder containing all videos: ").strip()

    # Loop through all videos in the folder and upload them
    for video_file in sorted(os.listdir(parent_folder)):
        video_path = os.path.join(parent_folder, video_file)

        if os.path.isfile(video_path) and video_file.endswith('.mp4'):
            print(f"🚀 Uploading video: {video_file}")
            upload_to_youtube(video_path, video_file, "Description for the video", parent_folder)

    print("✅ All videos processed and uploaded!")


🚀 Uploading video: Aldarayn.mp4
📤 Uploading: Aldarayn.mp4...
🔄 Token expired. Attempting to refresh...
✅ Token refreshed successfully!
✅ Upload complete! Video URL: https://www.youtube.com/watch?v=xC8xpMivCsI
🚀 Uploading video: Aldarayn_10.mp4
📤 Uploading: Aldarayn_10.mp4...
